# The key concept to master LCEL: the runnable execution order

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 003-runnable-exec-order.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **003-runnable-exec-order**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

## LCEL Chain
* **An LCEL Chain is a Sequence of Runnables.**
* Almost any component in LangChain (prompts, models, output parsers, vector store retrievers, tools, etc) can be used as a Runnable.
* Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves.
* The order (left to right) of the elements in a LCEL chain matters.

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he has a unique habit of sleeping in a very specific manner that he believes contributes to his performance. He reportedly takes multiple short naps throughout the day, totaling around 8 hours of sleep. Moreover, he often sleeps with the curtains drawn, ensuring complete darkness to enhance his sleep quality. This focus on sleep and recovery plays an important role in maintaining his high level of performance on the field.'

* All the components of the chain are Runnables.
* **When we write chain.invoke() we are using invoke with all the componentes of the chain in an orderly manner**:
    * First, we apply .invoke() with the user input to the prompt template.
    * Then, with the completed prompt, we apply .invoke() to the model.
    * And finally, we apply .invoke() to the output parser with the output of the model.
* IMPORTANT: the order of operations in a chain matters. If you try to execute the previous chain with the components in different order, the chain will fail.

## Let's see this graphically

![alt text](./lcel-2.png)


## And now let's replicate this process without using the LCEL chain

#### First, we apply .invoke() with the user input to the prompt template.

In [5]:
prompt.invoke({"soccer_player": "Ronaldo"})

ChatPromptValue(messages=[HumanMessage(content='tell me a curious fact about Ronaldo', additional_kwargs={}, response_metadata={})])

#### Then, with the completed prompt, we apply .invoke() to the model.

In [13]:
from langchain_core.messages.human import HumanMessage

output_after_first_step = [HumanMessage(content='tell me a curious fact about El Harapos Morales')]

In [14]:
model.invoke(output_after_first_step)

AIMessage(content='El Harapos Morales is a character from the Mexican folklore known for being a symbol of social commentary on poverty and marginalization. He is often depicted as a poor, tattered man who represents the struggles of the lower class in society. One curious fact about El Harapos Morales is that he often uses humor and satire to address serious issues, making him a complex figure. This duality allows him to critique societal norms while still being an endearing character in Mexican culture, showcasing the resilience and spirit of those who navigate hardships. His stories are often shared in various forms of media, including literature and theater, emphasizing the importance of tradition in Mexican storytelling.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 17, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_predict

#### And finally, we apply .invoke() to the output parser with the output of the model.

In [8]:
from langchain_core.messages.ai import AIMessage

output_after_second_step = AIMessage(content='One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9812511-043a-458a-bfb8-005bc0d057fb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

In [9]:
output_parser.invoke(output_after_second_step)

'One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.'

## Ways to execute Runnables
* Remember:
    * An LCEL Chain is a Sequence of Runnables.
    * Almost any component in LangChain (prompts, models, output parsers, etc) can be used as a Runnable.
    * Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves.
    * The order (left to right) of the elements in a LCEL chain matters.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 003-runnable-exec-order.py
* In terminal, make sure you are in the directory of the file and run:
    * python 003-runnable-exec-order.py